using the in-built model in pytorch

on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer as light_trainer

# device = torch.device('cuda')


/home/tian/anaconda3/envs/d2l/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# input_size = 784    #28 x 28
hidden_size = 128
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# using RNN to classify images means to input a image row by row, treat the rows as a seq.
input_size = 28
sequence_length = 28
num_layers = 2

In [3]:
# load datasets

# train_dataset = torchvision.datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=False)

test_dataset = torchvision.datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor())

In [4]:
# train_dataset.data.shape

In [5]:
# create data loader

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# rnn model
class RNN(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # treat the batch_size as the first dim
        # x.shape -> (batch_size, seq, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # rnn need 2 inputs: x and init_hidden
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(torch.device('cuda'))

        out, _ = self.rnn(x, h0)
        # out: batch_size, seq_length, hidden_size
        # decode the hidden_state only for the last step
        # out: N, 28, 128
        out = out[:, -1, :]
        # out: N, 128
        out = self.fc(out)
        return out

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer

    def training_step(self, batch):
        # rnn need 2 inputs: x and init_hidden
        images, labels = batch
        images = images.reshape(-1, sequence_length, input_size)
        labels = labels

        out = self(images)

        loss = self.criterion(out, labels)

        return {'loss': loss}
    
    def train_dataloader(self):
        train_dataset = torchvision.datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=False)

        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
        return train_loader


In [7]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

# n_total_steps = len(train_loader)

In [8]:
# training loop

# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader): 
#         # batch_size, 28, 28, doesn't need a reshape
#         images = images.reshape(-1, sequence_length, input_size).to(device)
#         labels = labels.to(device)

#         #forward pass
#         output = rnn(images)
#         loss = criterion(output, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if (i+1) % batch_size == 0:
#             print(f'Epoch {epoch+1} / {num_epochs}, Step [{i+1} / {n_total_steps}], LOss: {loss.item():.4f}')


In [14]:
torch.set_float32_matmul_precision('high')
model = RNN(input_size, hidden_size, num_layers, num_classes)
trainer = light_trainer(auto_lr_find=True, max_epochs=num_epochs,fast_dev_run=False, accelerator='gpu', devices=1)  # fast_dev_run will run a single batch to training, to test the model
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | rnn       | RNN              | 53.2 K
2 | fc        | Linear           | 1.3 K 
-----------------------------------------------
54.5 K    Trainable params
0         Non-trainable params
54.5 K    Total params
0.218     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [16]:
device = torch.device('cuda')

In [19]:
# eval loop
# prediction
model.to(device)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)

        # max returns (value, index)
        _, preds = torch.max(outputs, 1)
        n_samples += labels.size(0) # labels: [batch_size, 1]
        n_correct += (preds == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = preds[i]
            if label == pred :
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {test_dataset.classes[i]}: {acc} %')

Accuracy: 94.04 %
Accuracy of 0 - zero: 97.44897959183673 %
Accuracy of 1 - one: 99.20704845814979 %
Accuracy of 2 - two: 95.63953488372093 %
Accuracy of 3 - three: 94.65346534653466 %
Accuracy of 4 - four: 89.0020366598778 %
Accuracy of 5 - five: 85.53811659192826 %
Accuracy of 6 - six: 94.05010438413362 %
Accuracy of 7 - seven: 97.76264591439688 %
Accuracy of 8 - eight: 90.65708418891171 %
Accuracy of 9 - nine: 94.54905847373637 %
